Stippling Sandbox

In [27]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Sort_Vertices, Weighted_Raster_Centroid, Raster_BBox

In [28]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

Image parameters

In [29]:
image_resolution = 500
image_path = 'sampling/test_im_500.png'
# image_path = 'sphere.jpg'
weight_image = Image.open(image_path)
# weight_image = weight_image.filter(ImageFilter.BoxBlur(2))
res = image_resolution
jitter_mag = 5

scale_factor = image_resolution / res
# img = Image.new('RGB', (image_resolution, image_resolution))
img = Image.open(image_path)
img = weight_image

In [30]:
def Lloyd_Relax(seeds, image_resolution, image_array):
    vor = Voronoi(seeds)
    boundary_seeds = [
        np.array([-image_resolution, -image_resolution]),
        np.array([2 * image_resolution, -image_resolution]),
        np.array([2 * image_resolution, 2 * image_resolution]),
        np.array([-image_resolution, 2 * image_resolution])
    ]

    polygons = []
    open_cells = []
    for index, region in enumerate(vor.regions):
        poly = []
        if len(region) == 0:
            continue
        elif -1 in region:
            open_cells.append(region)
            continue
        for index in region:
            if index != -1:
                poly.append(vor.vertices[index])
            else:
                print("OUTSIDE")
        
        sorted_poly = Sort_Vertices(poly)

        polygons.append(sorted_poly)

    centroids = []
    for polygon in polygons:

        bbox = Raster_BBox(polygon, image_resolution)
        rastered_polygon = Scanline_Rasterize_Polygon(polygon, bbox, image_resolution)
        centroid = Weighted_Raster_Centroid(rastered_polygon, image_array)
        if centroid:
            if centroid[0] < 0:
                centroid[0] = 0
            elif centroid[0] > image_resolution:
                centroid[0] = image_resolution
            if centroid[1] < 0:
                centroid[1] = 0
            elif centroid[1] > image_resolution:
                centroid[1] = image_resolution

            centroids.append(centroid)
    
    new_seeds = boundary_seeds + centroids
    # new_seeds = centroids + open_cells

    for seed in new_seeds:
        x_jitter = (np.random.rand() - 0.5) * jitter_mag
        y_jitter = (np.random.rand() - 0.5) * jitter_mag
        seed = [seed[0] + x_jitter, seed[1] + y_jitter]
    print(len(open_cells))
    return new_seeds

Creating seeds

In [31]:
seeds = [
    np.array([-image_resolution, -image_resolution]),
    np.array([2 * image_resolution, -image_resolution]),
    np.array([2 * image_resolution, 2 * image_resolution]),
    np.array([-image_resolution, 2 * image_resolution])
]
startseeds = seeds
point_count = 1000

## Rejection sampling
# threshold = 150
threshold = 256

n = 0
while n < point_count:
    rand_x = np.random.rand() * image_resolution
    rand_y = np.random.rand() * image_resolution
    sampled_value = weight_image.getpixel((rand_x,rand_y))
    if np.mean(sampled_value) < threshold:
        seeds.append(np.array([rand_x,rand_y]))
        n += 1
    # seeds.append(np.array([rand_x,rand_y]))
    # n += 1
## Seed image
seed_img = Image.open(image_path)
draw = ImageDraw.Draw(seed_img)
for seed in startseeds:
    center_ellipse(seed[0], seed[1],2,'red')

seed_img.save("seed_image.jpg")

In [35]:
image_array = np.array(weight_image)
weight_array = np.zeros([500,500])
pixels = 0
for x in range(image_resolution):
    for y in range(image_resolution):
        pixel = image_array[x,y]
        weight_array[x,y] = np.mean(pixel) / 255

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [33]:
img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)
# for seed in seeds:
#     center_ellipse(seed[0], seed[1],2,'white')
#     img.save(f'sequence/stipple_result_{0}.png')

print("starting relaxation")
for i in range(30):
    new_seeds = Lloyd_Relax(seeds,res, weight_array)

    seeds = new_seeds
    print(f"Iteration {i}")

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)
center_rectangle(image_resolution / 2,image_resolution / 2,image_resolution,image_resolution,'white')
for seed in seeds:
    center_ellipse(seed[0], seed[1],2,'black')
    img.save(f'sequence/stipple_result.png')


starting relaxation
4
Iteration 0
4
Iteration 1
4
Iteration 2
4
Iteration 3
4
Iteration 4
4
Iteration 5
4
Iteration 6
4
Iteration 7
4
Iteration 8
4
Iteration 9
4
Iteration 10
4
Iteration 11
4
Iteration 12
4
Iteration 13
4
Iteration 14
4
Iteration 15
4
Iteration 16
4
Iteration 17
4
Iteration 18
4
Iteration 19
4
Iteration 20
4
Iteration 21
4
Iteration 22
4
Iteration 23
4
Iteration 24
4
Iteration 25
4
Iteration 26
4
Iteration 27
4
Iteration 28
4
Iteration 29


In [34]:
img = Image.new('RGB', (image_resolution * 2, image_resolution * 2))
draw = ImageDraw.Draw(img)
center_rectangle(image_resolution,image_resolution,image_resolution * 2,image_resolution * 2,'white')
for seed in seeds:
    center_ellipse(seed[0] * 2, seed[1] * 2,2,'black')
    img.save(f'sequence/stipple_result_scaled.png')